In [1]:
import pandas as pd
import time
import re
import pathlib
import os
import sys
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [2]:
df = pd.read_csv('Distrito8_2020_Pleb.csv')

In [3]:
%%time
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')

driver_path = 'C:\\Users\\Chandrasekhar\\Downloads\\chromedriver.exe'

driver = webdriver.Chrome(executable_path = driver_path, options=options)

driver.get('https://pv.servelelecciones.cl/')

#constpol = '/html/body/div[2]/div[1]/div[1]/menu-elecciones/div/nav/div/div[2]/ul/li[1]/a'
#tipoorgano = '/html/body/div[2]/div[1]/div[1]/menu-elecciones/div/nav/div/div[2]/ul/li[2]/a'
#participacion = '/html/body/div[2]/div[1]/div[1]/menu-elecciones/div/nav/div/div[2]/ul/li[3]/a'
#instalmesas = '/html/body/div[2]/div[1]/div[1]/menu-elecciones/div/nav/div/div[2]/ul/li[4]/a'

#PESTAÑA SUPERIOR
step1 = '/html/body/div[2]/div[1]/div[1]/menu-elecciones/div/nav/div/div[2]/ul/li[1]/a'
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, step1))).click()
time.sleep(2)

#NACIONAL
step2 = '/html/body/div[2]/div[1]/div[2]/div/ul/li[3]/a'
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, step2))).click()
time.sleep(2)


#DISTRITO
step3 = '/html/body/div[2]/div[1]/div[2]/div/form/select[6]/option[9]'
WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, step3))).click()
time.sleep(3)


def Comuna(df):
    return df['COMUNA'].unique()

def Circuns(df, i):
    return df.loc[(df['COMUNA']==Comuna(df)[i])]['CIRCUNSCRIPCION'].unique()

def Colegio(df, i, j):
    return df.loc[(df['COMUNA']==Comuna(df)[i]) & (df['CIRCUNSCRIPCION']==Circuns(df, i)[j])]['COLEGIO'].unique()

def Mesa(df, i, j, k):
    return df.loc[(df['COMUNA']==Comuna(df)[i]) & (df['CIRCUNSCRIPCION']==Circuns(df, i)[j]) & (df['COLEGIO']==Colegio(df, i, j)[k])]['MESA'].unique()

#COMUNA
for m in range(len(Comuna(df))):
    time.sleep(6)
    step4 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[9]/option[{m+2}]'
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, step4))).click()
    #CIRCUNSCRIPCION ELECTORAL
    for j in range(len(Circuns(df, m))):
        time.sleep(6)
        step5 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[10]/option[{j+2}]'
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, step5))).click()
        #COLEGIO
        for k in range(len(Colegio(df, m, j))):
            step6 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[11]/option[{k+2}]'
            WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, step6))).click()
            time.sleep(6)
            #MESA
            for q in range(len(Mesa(df, m, j, k))):
                step7 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[12]/option[{q+2}]'
                WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, step7))).click()        
                time.sleep(2)
                df3 = pd.DataFrame(re.split("(\n)", driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[4]/dynamic-table/div/table/tbody[1]').text))
                df3 = df3[df3 != '\n']       
                df3 = df3.dropna(axis=0).reset_index(drop=True)
                df3[0] = df3[0].apply(lambda x: "".join(re.split("( )", x)))
                df3 = df3[0].str.split(expand=True)
                df.loc[(df['MESA'] == Mesa(df, m, j, k)[q]) & (df['CIRCUNSCRIPCION'] == Circuns(df, m)[j]), 'APRUEBO'] = df3[1][0]
                df.loc[(df['MESA'] == Mesa(df, m, j, k)[q]) & (df['CIRCUNSCRIPCION'] == Circuns(df, m)[j]), 'RECHAZO'] = df3[1][1]

Wall time: 2h 9min 23s


In [51]:
df['APRUEBO'] = df['APRUEBO'].astype(int)
df.loc[(df['RECHAZO']=='0,00%')&(df['COLEGIO']=='LICEO MONSEÑOR ENRIQUE ALVEAR LOCAL: 2'),'RECHAZO'] = 0
df['RECHAZO'] = df['RECHAZO'].astype(int)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3132 entries, 0 to 3131
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   MESA             3132 non-null   object 
 1   TOTALMESAS       3132 non-null   int64  
 2   ELECTORES        3132 non-null   int64  
 3   EMITIDOS         3132 non-null   int64  
 4   PORCENTAJE       3132 non-null   float64
 5   COLEGIO          3132 non-null   object 
 6   COMUNA           3132 non-null   object 
 7   CIRCUNSCRIPCION  3132 non-null   object 
 8   APRUEBO          3132 non-null   int32  
 9   RECHAZO          3132 non-null   int32  
dtypes: float64(1), int32(2), int64(3), object(4)
memory usage: 220.3+ KB


In [103]:
df.to_csv('Distrito8_2020_Pleb_Resultados.csv', index=False)

In [89]:
df = pd.read_csv('Distrito8_2020_Pleb_ResultadosTEST1.csv')

In [90]:
df['NULOSBLANCOS'] = df['EMITIDOS'] - df['APRUEBO'] - df['RECHAZO']

In [102]:
len(df[df['NULOSBLANCOS']<0])

28

In [101]:
df[df['NULOSBLANCOS']<0]

,MESA,TOTALMESAS,ELECTORES,EMITIDOS,PORCENTAJE,COLEGIO,COMUNA,CIRCUNSCRIPCION,APRUEBO,RECHAZO,NULOSBLANCOS
32,80M,1,329,-1,-0.30,ESCUELA BASICA CAMPOS DE MAIPU,CERRILLOS,CERRILLOS,177,27,-205
51,39V,1,332,183,55.12,ESCUELA BASICA PEDRO AGUIRRE CERDA,CERRILLOS,CERRILLOS,165,20,-2
62,50V,1,327,180,55.05,ESCUELA BASICA PEDRO AGUIRRE CERDA,CERRILLOS,CERRILLOS,159,23,-2
84,33M,1,339,158,46.61,ESCUELA CERRILLOS,CERRILLOS,CERRILLOS,138,27,-7
89,38M,1,329,161,48.94,ESCUELA CERRILLOS,CERRILLOS,CERRILLOS,142,19,-21
249,39,1,278,-1,-0.36,ESCUELA SAN FRANCISCO DE ASIS,COLINA,CHICUREO,71,45,-117
251,103,1,337,171,50.74,ESCUELA ANDALIEN,COLINA,COLINA,137,54,-20
260,112,1,332,165,49.70,ESCUELA ANDALIEN,COLINA,COLINA,122,46,-3
611,51V,1,339,172,50.74,ESCUELA D-276 CAROLINA VERGARA AYARES,ESTACION CENTRAL,ALAMEDA,142,32,-2
654,112,1,337,181,53.71,ESCUELA REPUBLICA DE PALESTINA,ESTACION CENTRAL,ALAMEDA,175,16,-10


In [48]:
df.loc[(df['RECHAZO']=='0,00%')]

,MESA,TOTALMESAS,ELECTORES,EMITIDOS,PORCENTAJE,COLEGIO,COMUNA,CIRCUNSCRIPCION,APRUEBO,RECHAZO


In [83]:
#df[df['COLEGIO']=='COLEGIO POLIVALENTE GERONIMO DE ALDERETE']

In [100]:
#SI HAY ALGUN COLEGIO CON DATOS ERRADOS SE DEBE MODIFICAR ESTA PARTE PARA ARREGLAR!
for q in range(0,19):
    step7 = f'/html/body/div[2]/div[1]/div[2]/div/form/select[12]/option[{q+2}]'
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, step7))).click()        
    time.sleep(3)
    df3 = pd.DataFrame(re.split("(\n)", driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[4]/dynamic-table/div/table/tbody[1]').text))
    df3 = df3[df3 != '\n']       
    df3 = df3.dropna(axis=0).reset_index(drop=True)
    df3[0] = df3[0].apply(lambda x: "".join(re.split("( )", x)))
    df3 = df3[0].str.split(expand=True)
    df.loc[(df['MESA'] == Mesa(df, 0, 0, 4)[q]) & (df['CIRCUNSCRIPCION'] == Circuns(df, 0)[0]), 'APRUEBO'] = df3[1][0]
    df.loc[(df['MESA'] == Mesa(df, 0, 0, 4)[q]) & (df['CIRCUNSCRIPCION'] == Circuns(df, 0)[0]), 'RECHAZO'] = df3[1][1]

In [99]:
#Comuna(df)
#Circuns(df,0)[0]
#Colegio(df,0,0)[4]
len(Mesa(df, 0, 0, 4))

19

In [71]:
for q in range(0,24):
    print(q)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
